# Calculation of mixing diagnostics, save yearly mean of diapycnal transport

In [1]:
%matplotlib inline
import xarray as xr
import numpy as np
import cosima_cookbook as cc
from collections import OrderedDict
from dask.distributed import Client
import matplotlib.path as mpath

import cf_xarray
from metpy.interpolate import cross_section
import pyproj

import matplotlib.pyplot as plt
import cmocean.cm as cmo
import matplotlib.colors as col
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter
import matplotlib.ticker as mticker

In [2]:
def yearly_mean(var):
    # construct an xarray of days per month
    month_length = var.time.dt.days_in_month
    weights_month = (month_length.groupby('time.year') /
                     month_length.groupby('time.year').sum())
    var = (var * weights_month).groupby('time.year').sum()
    var = var.rename({'year': 'time'})
    var = var.where(var != 0)
    return var

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/36993/status,
Dashboard: /proxy/36993/status,Workers: 7
Total threads: 28,Total memory: 251.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38063,Workers: 7
Dashboard: /proxy/36993/status,Total threads: 28
Started: Just now,Total memory: 251.20 GiB
Comm: tcp://127.0.0.1:46371,Total threads: 4
Dashboard: /proxy/39203/status,Memory: 35.89 GiB
Nanny: tcp://127.0.0.1:33707,


In [4]:
DSW_region = {
    'name': ['Weddell', 'Prydz', 'Adelie', 'Ross'],
    'lon_min_area': [-58, 47, 90-360, 166-360],
    'lon_max_area': [-30, 72, 147-360, -170],
    'lat_min_area': [-75, -68, -67.5, -76.5],
    'lat_max_area': [-59, -64, -61.9, -65.8]}

In [9]:
session = cc.database.create_session()
expt = 'panant-005-zstar-ACCESSyr2'
expt_name = 'panan_005deg_jra55_ryf'
resolution = expt_name.split('_')[1][:-3]

year = '2005'
start_time= year + '-01-01'
end_time= year + '-12-31'

frequency = '1 monthly'
path_output = '/g/data/e14/cs6673/mom6_comparison/data_DSW/'

## Load data to calculate and save age, rho and mixing diagnostics as yearly means

In [ ]:
%%time
sig_min = 1035

rho = cc.querying.getvar(
    expt, 'rhopot2', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'xh': '200MB', 'yh': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55))
rho = yearly_mean(rho).mean('time').compute()

age = cc.querying.getvar(
    expt, 'agessc', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'xh': '200MB', 'yh': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55))
age = yearly_mean(age).mean('time').compute()

# thickness of layers
area = cc.querying.getvar(
    expt, 'areacello', session, n=1,
    chunks={'xh': '200MB', 'yh': '200MB'}).sel(
    yh=slice(None, -55))
vol = cc.querying.getvar(
    expt, 'volcello', session,
    frequency='1 monthly',
    attrs={'cell_methods': 'area:sum rho2_l:sum yh:sum xh:sum time: mean'} ,
    start_time=start_time, end_time=end_time,
    chunks={'rho2_l': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55),
    rho2_l=slice(sig_min, None))
hmo = vol/area
hmo = yearly_mean(hmo.cumsum('rho2_l')).mean('time').compute()

# mixing diagnostics
Kd_heat = cc.querying.getvar(
    expt, 'Kd_heat', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'rho2_i': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55),
    rho2_i=slice(sig_min, None))
Kd_heat = yearly_mean(Kd_heat).mean('time').compute()

Kd_salt = cc.querying.getvar(
    expt, 'Kd_salt', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'rho2_i': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55),
    rho2_i=slice(sig_min, None))
Kd_salt = yearly_mean(Kd_salt).mean('time').compute()

Kd_shear = cc.querying.getvar(
    expt, 'Kd_shear', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'rho2_i': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55),
    rho2_i=slice(sig_min, None))
Kd_shear = yearly_mean(Kd_shear).mean('time').compute()

Kd_BBL = cc.querying.getvar(
    expt, 'Kd_BBL', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'rho2_i': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55),
    rho2_i=slice(sig_min, None))
Kd_BBL = yearly_mean(Kd_BBL).mean('time').compute()

Kd_ePBL = cc.querying.getvar(
    expt, 'Kd_ePBL', session, frequency=frequency,
    start_time=start_time, end_time=end_time,
    chunks={'rho2_i': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55),
    rho2_i=slice(sig_min, None))
Kd_ePBL = yearly_mean(Kd_ePBL).mean('time').compute()

In [52]:
rho.name = 'rhopot2'
ds_z = rho.to_dataset()
ds_z['agessc'] = age
ds_z = ds_z.assign_coords(time=[np.int(year)])

Kd_heat.name = 'Kd_heat'
ds_rho = Kd_heat.to_dataset()
ds_rho['Kd_salt'] = Kd_salt
ds_rho['Kd_shear'] = Kd_shear
ds_rho['Kd_BBL'] = Kd_BBL
ds_rho['Kd_ePBL'] = Kd_ePBL
ds_rho['hmo'] = hmo
ds_rho = ds_rho.assign_coords(time=[np.int(year)])

In [53]:
%%time
"""save data"""
comp = dict(chunksizes=(42, 292, 1200),
            zlib=True, complevel=5, shuffle=True)
enc_rho = {var: comp for var in ds_rho.data_vars}
enc_z = {var: comp for var in ds_z.data_vars}
ds_z.to_netcdf(
    path_output + 'Age_rhopot2_' + expt_name + '_' +
    year + '.nc', encoding=enc_z)


CPU times: user 2min 48s, sys: 18.5 s, total: 3min 6s
Wall time: 3min 4s


In [54]:
%%time
ds_rho.to_netcdf(
    path_output + 'mixing_diagnostics_' + expt_name + '_' +
    year + '.nc', encoding=enc_rho)

CPU times: user 6min 21s, sys: 44.6 s, total: 7min 5s
Wall time: 6min 59s


## Load data to cut out age, rho, layer thickness and diapycnal mixing in DSW regions and save as monthly means

In [39]:
%%time
for year in range(2003, 2006):
    for a, area_text in enumerate(DSW_region['name']):
        print(a)
        rho = cc.querying.getvar(
            expt, 'rhopot2', session, frequency=frequency,
            start_time=start_time, end_time=end_time,
            chunks={'xh': '200MB', 'yh': '200MB'}).sel(
            time=slice(start_time, end_time)).assign_coords(
            {'area': area_text})
        rho = rho.sel(
            xh=slice(DSW_region['lon_min_area'][a],
                     DSW_region['lon_max_area'][a]),
            yh=slice(DSW_region['lat_min_area'][a],
                     DSW_region['lat_max_area'][a])).compute()
        
        age = cc.querying.getvar(
            expt, 'agessc', session, frequency=frequency,
            start_time=start_time, end_time=end_time,
            chunks={'xh': '200MB', 'yh': '200MB'}).sel(
            time=slice(start_time, end_time)).assign_coords(
            {'area': area_text})
        age = age.sel(
            xh=slice(DSW_region['lon_min_area'][a],
                     DSW_region['lon_max_area'][a]),
            yh=slice(DSW_region['lat_min_area'][a],
                     DSW_region['lat_max_area'][a])).compute()
    
        ds_z = rho.to_dataset()
        ds_z['agessc'] = age
    
        """save data"""
        comp = dict(chunksizes=(12, 75, 90, 200),
                    zlib=True, complevel=5, shuffle=True)
        enc_z = {var: comp for var in ds_z.data_vars}
        ds_z.to_netcdf(
            path_output + 'Age_rhopot2_in_' + area_text + '_' +
            expt_name + '_1m_' + year + '.nc', encoding=enc_z)

0
1
2
3
CPU times: user 2min 27s, sys: 22.2 s, total: 2min 49s
Wall time: 3min 39s


In [49]:
%%time
for year in range(2003, 2006):
    for a, area_text in enumerate(DSW_region['name']):
        print(a)
        ds = xr.open_mfdataset(
            path_output + 'Diapycnal_transport_at_upper_interface_' +
            expt_name + '_' + frequency[:3:2] + '_' +
            str(year) + '*nc', concat_dim='time', combine='nested')
        ds = ds.sel(
            xh=slice(DSW_region['lon_min_area'][a],
                     DSW_region['lon_max_area'][a]),
            yh=slice(DSW_region['lat_min_area'][a],
                     DSW_region['lat_max_area'][a])).compute()
        enc = {'diapycnal_transport':
               {'chunksizes': (len(ds.time), 99, 90, 200),
                'zlib': True, 'complevel': 5, 'shuffle': True}}
        
        ds.to_netcdf(
            path_output + 'Diapycnal_transport_at_upper_interface_' +
            'in_' + area_text + '_' + expt_name + '_1m_' + str(year) +
            '.nc', encoding=enc)

0
1
2
3
0
1
2
3
0
1
2
3
CPU times: user 8min 30s, sys: 1min 25s, total: 9min 56s
Wall time: 12min 4s


In [10]:
%%time
for year in range(2003, 2006):
    for a, area_text in enumerate(DSW_region['name']):
        print(a)
        start_time = str(year) + '-01-01'
        end_time = str(year) + '-12-31'

        # thickness of layers
        area = cc.querying.getvar(
            expt, 'areacello', session, n=1,
            chunks={'xh': '200MB', 'yh': '200MB'}).sel(
            yh=slice(None, -55))
        vol = cc.querying.getvar(
            expt, 'volcello', session,
            frequency='1 monthly',
            attrs={'cell_methods': 'area:sum rho2_l:sum yh:sum xh:sum time: mean'} ,
            start_time=start_time, end_time=end_time,
            chunks={'rho2_l': '200MB'}).sel(
            time=slice(start_time, end_time), yh=slice(None, -55))
        hmo = vol/area
        hmo = hmo.sel(
            xh=slice(DSW_region['lon_min_area'][a],
                     DSW_region['lon_max_area'][a]),
            yh=slice(DSW_region['lat_min_area'][a],
                     DSW_region['lat_max_area'][a]))
        hmo = hmo.cumsum('rho2_l').compute()
        
        hmo.name = 'hmo'
        enc = {'hmo':
               {'chunksizes': (len(hmo.time), 99, 90, 200),
                'zlib': True, 'complevel': 5, 'shuffle': True}}
        hmo.to_netcdf(
            path_output + 'Layer_thickness_' +
            'in_' + area_text + '_' + expt_name + '_1m_' + str(year) +
            '.nc', encoding=enc)

0


Exception during reset or similar
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 763, in _finalize_fairy
    fairy._reset(pool, transaction_was_reset)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 1038, in _reset
    pool._dialect.do_rollback(self)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 683, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 22438727538496 and this is thread id 22434082187008.
Exception closing connection <sqlite3.Connection object at 0x1467aefaaa40>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/sit

1
2
3
0
1
2
3
0
1
2
3
CPU times: user 7min 7s, sys: 55.2 s, total: 8min 2s
Wall time: 8min 27s


### diapycnal transport: save as yearly means

files with monthly data were calculated using run_diapycnal_transp_calculation.sh

In [7]:
expt = 'panant-005-zstar-ACCESSyr2'
expt_name = 'panan_005deg_jra55_ryf'
resolution = expt.split('-')[1]

frequency = '1 monthly'
path_output = '/g/data/e14/cs6673/mom6_comparison/data_DSW/'

resolution

'005'

In [8]:
%%time
for year in range(2003, 2006):
    ds = xr.open_mfdataset(path_output + 'Diapycnal_transport_at_upper_interface_' +
                expt_name + '_' + frequency[:3:2] + '_' +
                str(year) + '*nc', concat_dim='time', combine='nested')
    ds_mean = yearly_mean(ds).squeeze().compute()
    enc = {'diapycnal_transport':
           {'chunksizes': (50, 292, 1200),
            'zlib': True, 'complevel': 5, 'shuffle': True}}
    
    ds_mean.to_netcdf(path_output + 'Diapycnal_transport_at_upper_interface_' +
                      expt_name + '_1y_' + str(year) + '.nc', encoding=enc)

CPU times: user 7min 31s, sys: 1min 1s, total: 8min 33s
Wall time: 11min 47s


In [9]:
ds_mean

<xarray.Dataset>
Dimensions:              (rho2_l: 99, yh: 1166, xh: 7200)
Coordinates:
  * xh                   (xh) float64 -280.0 -279.9 -279.9 ... 79.87 79.92 79.97
  * yh                   (yh) float64 -81.1 -81.08 -81.06 ... -55.05 -55.02
  * rho2_l               (rho2_l) float64 1.016e+03 1.033e+03 ... 1.038e+03
    time                 int64 2005
Data variables:
    diapycnal_transport  (rho2_l, yh, xh) float64 nan nan nan ... nan nan nan

### diapycnal transport: calculate for individual months/years

This can be run in parallel using run_diapycnal_transp_calculation.sh

In [5]:
year = 2004
month = 1

In [6]:
if resolution == '01':
    start_time = str(year) + '-01-01'
    end_time = str(year+1) + '-01-02'
elif resolution == '005':
    if month == 12:
        start_time = str(year) + '-' +  str(month).zfill(2) + '-01'
        end_time = str(year+1) + '-01-02'
    else:
        start_time = str(year) + '-' +  str(month).zfill(2) + '-01'
        end_time = str(year) + '-' +  str(month+1).zfill(2) + '-02'

In [7]:
# UMO and VMO
U = cc.querying.getvar(
    expt, 'umo', session, frequency='1 monthly',
    start_time=start_time, end_time=end_time,
    chunks={'rho2_l': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55)).squeeze()
V = cc.querying.getvar(
    expt, 'vmo', session, frequency='1 monthly',
    start_time=start_time, end_time=end_time,
    chunks={'rho2_l': '200MB'}).sel(
    time=slice(start_time, end_time), yq=slice(None, -55)).squeeze()

vol = cc.querying.getvar(
    expt, 'volcello', session,
    attrs={'cell_methods': 'area:sum rho2_l:sum yh:sum xh:sum time: point'} ,
    start_time=start_time, end_time=end_time,
    chunks={'rho2_l': '200MB'}).sel(
    time=slice(start_time, end_time), yh=slice(None, -55))
# change in volume per second between monthly snapshots * density
dvol =  vol.diff('time', label='lower')/(
    vol.time.diff('time', label='lower').astype('int')/1e9)*vol.rho2_l
dvol = dvol.squeeze()

In [8]:
def transport_across_isopycnals_12months(expt, U, V, dvol):
    resolution = expt.split('-')[1]
    if resolution == '01':
        U = U.isel(yh=slice(None, -1))
        dvol = dvol.isel(yh=slice(None, -1))
        if str(U.time[0].values)[:7] == '2003-01':
            U = U[1:, :]
            V = V[1:, :]

    D = 0*dvol 
    k = len(dvol.rho2_l)-1
    D[:, k, :] = (dvol.isel(rho2_l=k) -
                  (U.isel(xq=slice(1, None), rho2_l=k).values -
                   U.isel(xq=slice(None, -1), rho2_l=k).values) -
                  (V.isel(yq=slice(1, None), rho2_l=k).values -
                   V.isel(yq=slice(None, -1), rho2_l=k).values))
    for k in range(len(dvol.rho2_l)-2, -1, -1):
        D[:, k, :] = (dvol.isel(rho2_l=k) + D[:, k+1, :] -
                      (U.isel(xq=slice(1, None), rho2_l=k).values -
                       U.isel(xq=slice(None, -1), rho2_l=k).values) -
                      (V.isel(yq=slice(1, None), rho2_l=k).values -
                       V.isel(yq=slice(None, -1), rho2_l=k).values))
    D['time'] = U.time
    return D

In [23]:
def transport_across_isopycnals_1month(expt, U, V, dvol):
    resolution = expt.split('-')[1]
    if resolution == '01':
        U = U.isel(yh=slice(None, -1))
        dvol = dvol.isel(yh=slice(None, -1))

    D = 0*dvol 
    k = len(dvol.rho2_l)-1
    D[k, :] = (dvol.isel(rho2_l=k) -
                  (U.isel(xq=slice(1, None), rho2_l=k).values -
                   U.isel(xq=slice(None, -1), rho2_l=k).values) -
                  (V.isel(yq=slice(1, None), rho2_l=k).values -
                   V.isel(yq=slice(None, -1), rho2_l=k).values))
    for k in range(len(dvol.rho2_l)-2, -1, -1):
        D[k, :] = (dvol.isel(rho2_l=k) + D[k+1, :] -
                      (U.isel(xq=slice(1, None), rho2_l=k).values -
                       U.isel(xq=slice(None, -1), rho2_l=k).values) -
                      (V.isel(yq=slice(1, None), rho2_l=k).values -
                       V.isel(yq=slice(None, -1), rho2_l=k).values))
    D['time'] = U.time
    return D

In [ ]:
if resolution == '01':
    D = transport_across_isopycnals_12months(expt, U, V, dvol)
elif resolution == '005':
    D = transport_across_isopycnals_1month(expt, U, V, dvol)

In [25]:
D

<xarray.DataArray (rho2_l: 99, yh: 583, xh: 3600)>
dask.array<setitem, shape=(99, 583, 3600), dtype=float64, chunksize=(99, 121, 515), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95
  * yh       (yh) float64 -81.11 -81.07 -81.02 -80.98 ... -55.18 -55.12 -55.07
  * rho2_l   (rho2_l) float64 1.016e+03 1.033e+03 ... 1.037e+03 1.038e+03
    time     object 2004-01-16 12:00:00

In [ ]:
D.name = 'diapycnal_transport'
if resolution == '01':
    time_str = str(year)
    enc = {'diapycnal_transport':
           {'chunksizes': (1, 50, 292, 1200),
            'zlib': True, 'complevel': 5, 'shuffle': True}}
elif resolution == '005':
    time_str = str(D.time.values)[:7]
    enc = {'diapycnal_transport':
           {'chunksizes': (50, 292, 1200),
            'zlib': True, 'complevel': 5, 'shuffle': True}}

D.to_netcdf(path_output + 'Diapycnal_transport_at_upper_interface_' +
            expt_name + '_' + frequency[:3:2] + '_' +
            time_str + '_test.nc', encoding=enc)